# Regression
Features = rates from selected diseases (FeatureSelection) + suicide rate 

Target = suicide rate of the following year

Import libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
root = "../../"

Prepare dataset

In [5]:
disease = ""
path = root + "CSV/TabNet/Internacoes_Rate/"

suicide = pd.read_csv(root +'CSV/Suicide/suicide_rates_08_18.csv', index_col=0)

years = ["08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18"]

final_df = pd.DataFrame()
selected_diseases = ["UROLITÍASE", 
                     "TRANST_MENTAIS_COMPORT_DEV_USO_OUTR_SUBST_PSICOAT", 
                     "TRANSTORNOS_DE_HUMOR_[AFETIVOS]", 
                     "TRANSTORNOS_DE_CONDUÇÃO_E_ARRITMIAS_CARDÍACAS", 
                     "PARTO_ÚNICO_ESPONTÂNEO", 
                     "PANCREATITE_AGUDA_E_OUTRAS_DOENÇAS_DO_PÂNCREAS", 
                     "OUTROS_TRANSTRONOS_ARTICULARES",
                    "OUTRAS_NEOPLASIAS_MALIGNAS_DA_PELE",
                    "OUTRAS_DOENÇAS_DOS_INTESTINOS_E_PERITÔNIO",
                    "OUTRAS_ANEMIAS",
                    "NEOPL_MALIG_OUTR_LOCALIZ_MAL_DEF_SECUN_E_NÃO_ESPEC",
                    "NEOPL_MALIG_JUNÇÃO_RETOSSIGM_RETO_ÂNUS_CANAL_ANAL",
                    "NEOPLASIA_MALIGNA_DO_PÂNCREAS",
                    "NEOPLASIA_MALIGNA_DO_ESÔFAGO",
                    "NEOPLASIA_MALIGNA_DO_CÓLON",
                    "NEOPLASIA_MALIGNA_DE_TRAQUÉIA_BRÔNQUIOS_E_PULMÕES",
                    "LEPTOSPIROSE_NÃO_ESPECIFICADA",
                    "INSUFICIÊNCIA_RENAL",
                    "INSUFICIÊNCIA_CARDÍACA",
                    "INFARTO_AGUDO_DO_MIOCÁRDIO",
                    "FRATURA_DO_PESCOÇO_TÓRAX_OU_PELVE", 
                     "FRATURA_DO_FÊMUR",
                    "DOENÇAS_RENAIS_TÚBULO-INTERSTICIAIS",
                    "DOENÇAS_DO_APÊNDICE",
                    "DIABETES_MELLITUS",
                    "COLELITÍASE_E_COLECISTITE",
                    "BRONQUITE_ENFISEMA_E_OUTR_DOENÇ_PULM_OBSTR_CRÔNIC"]

for i in range(1,len(years)-1):
    col_year_suicide = "RATE_" + years[i+1]
    col_year_prev = "RATE_" + years[i]
    col_year_prev_prev = "RATE_" + years[i-1]
    year_df = suicide[[col_year_prev,col_year_prev_prev,col_year_suicide, "MUNCOD"]].copy()
    year_df = year_df.rename(columns={col_year_suicide: "RATE"})
    year_df["PREVIOUS"] = (year_df[col_year_prev] + year_df[col_year_prev_prev])/2
    year_df = year_df.drop(columns=[col_year_prev,col_year_prev_prev])
    for disease in selected_diseases:
        col_year_disease = "RATE_" + years[i]
        disease_df = pd.read_csv(path + disease + ".csv", sep=',', index_col=0)
        disease_df = disease_df[[col_year_disease, "MUNCOD"]]
        disease_df = disease_df.rename(columns={col_year_disease: disease})
        
        year_df = pd.merge(disease_df, year_df, left_on="MUNCOD", right_on="MUNCOD", how='right')
        
    year_df = year_df.drop("MUNCOD", axis=1)
    final_df = pd.concat([final_df, year_df])

final_df = final_df[(final_df["RATE"] > 0) | (final_df["PREVIOUS"] > 0)]
final_df = final_df.fillna(0)
final_df.head()

,BRONQUITE_ENFISEMA_E_OUTR_DOENÇ_PULM_OBSTR_CRÔNIC,COLELITÍASE_E_COLECISTITE,DIABETES_MELLITUS,DOENÇAS_DO_APÊNDICE,DOENÇAS_RENAIS_TÚBULO-INTERSTICIAIS,FRATURA_DO_FÊMUR,FRATURA_DO_PESCOÇO_TÓRAX_OU_PELVE,INFARTO_AGUDO_DO_MIOCÁRDIO,INSUFICIÊNCIA_CARDÍACA,INSUFICIÊNCIA_RENAL,...,OUTRAS_NEOPLASIAS_MALIGNAS_DA_PELE,OUTROS_TRANSTRONOS_ARTICULARES,PANCREATITE_AGUDA_E_OUTRAS_DOENÇAS_DO_PÂNCREAS,PARTO_ÚNICO_ESPONTÂNEO,TRANSTORNOS_DE_CONDUÇÃO_E_ARRITMIAS_CARDÍACAS,TRANSTORNOS_DE_HUMOR_[AFETIVOS],TRANST_MENTAIS_COMPORT_DEV_USO_OUTR_SUBST_PSICOAT,UROLITÍASE,RATE,PREVIOUS
0,205.305083,205.305083,221.729490,24.636610,110.864745,8.212203,8.212203,12.318305,328.488133,16.424407,...,8.212203,12.318305,4.106102,640.551860,32.848813,0.000000,0.000000,119.076948,8.189337,14.278214
1,129.762336,101.705615,100.536585,86.508224,115.733976,43.254112,3.507090,12.859331,143.790697,26.887691,...,0.000000,11.690301,3.507090,699.079973,4.676120,4.676120,0.000000,43.254112,4.427031,5.898225
2,164.301718,44.809559,224.047797,74.682599,418.222554,59.746079,14.936520,29.873040,59.746079,44.809559,...,0.000000,0.000000,0.000000,1015.683346,14.936520,14.936520,0.000000,74.682599,0.000000,7.468260
3,59.739434,78.805211,209.723546,59.739434,13.981570,15.252622,17.794725,21.607881,245.312997,80.076263,...,2.542104,8.897363,5.084207,613.918017,3.813155,1.271052,2.542104,25.421036,2.544497,6.368641
4,541.451089,162.435327,469.257610,78.209602,138.370834,18.048370,24.064493,18.048370,246.661052,54.145109,...,6.016123,0.000000,24.064493,776.079894,18.048370,0.000000,6.016123,138.370834,11.743981,0.000000


In [6]:
final_df.shape

(39120, 29)

Define X and y

In [7]:
X = final_df.drop(columns=["RATE"])
y = final_df["RATE"]

Split data into train and test sets

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [9]:
# Bayesian Optimization
def train_model(params):
    n_estimators = params[0]
    max_depth = params[1]
    min_samples_leaf = params[2]
    max_features = params[3]
    
    xf_train_params,xf_val,yf_train_params,yf_val = train_test_split(X_train,y_train,test_size=0.33)
    
    model = RandomForestRegressor(n_jobs=-1, random_state=42,n_estimators=n_estimators, 
                                  max_depth=max_depth,min_samples_leaf=min_samples_leaf,max_features=max_features)
    
    model.fit(xf_train_params,yf_train_params)
    yf_pred = model.predict(xf_val)
    
    return np.sqrt(metrics.mean_squared_error(yf_val, yf_pred))

space = [
    (5,500), #n_estimators
    (3,15), #max_depth
    (2,100), #min_samples_leaf
    (0.25,1.00) #max_features
]

from skopt import gp_minimize

resultado_gp = gp_minimize(train_model, space, random_state=42, verbose=0, n_calls=30, n_random_starts=10)

In [10]:
resultado_gp.x

[470, 3, 96, 0.7130775326589291]

Fit the RandomForest regression model

In [11]:
regressor = RandomForestRegressor(n_jobs=-1, random_state=42,n_estimators=resultado_gp.x[0], 
                                  max_depth=resultado_gp.x[1],min_samples_leaf=resultado_gp.x[2],max_features=resultado_gp.x[3])
regressor.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=3, max_features=0.7130775326589291,
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=96, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=470, n_jobs=-1,
                      oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

Calculate metrics corresponding to the regression model

In [12]:
def calcMetrics(y_train, y_pred_train, y_test, y_pred):
    print("Root Mean Squared Error Train: " + str(np.sqrt(metrics.mean_squared_error(y_pred_train,y_train))))
    print("Root Mean Squared Error Test: " + str(np.sqrt(metrics.mean_squared_error(y_test,y_pred))))
    #print("MAPE Test: " + str(np.mean(np.abs((y_test - y_pred) / y_test)) * 100))
def calcUpMetrics(y_pred,y_test,X_test):
    up_df = pd.DataFrame({"Pred": y_pred, "Real": y_test, "Previous": X_test["PREVIOUS"]})
    up_df["UP"] = up_df["Previous"] < up_df["Real"]
    up_df["UP_PRED"] = up_df["Previous"] < up_df["Pred"]
    up_df["UP"] = up_df["UP"].astype(int)
    up_df["UP_PRED"] = up_df["UP_PRED"].astype(int)
    print("Accuracy UP/DOWN: ", metrics.accuracy_score(up_df["UP"], up_df["UP_PRED"]))

In [13]:
y_pred = regressor.predict(X_test) 
y_pred_train = regressor.predict(X_train)
print("# Regression Model")
calcMetrics(y_train, y_pred_train, y_test, y_pred)
calcUpMetrics(y_pred,y_test,X_test)

# Regression Model
Root Mean Squared Error Train: 11.264555350506278
Root Mean Squared Error Test: 11.240287391770098
Accuracy UP/DOWN:  0.674593338497289


We also define a baseline model consisting of predicting the previous year suicide rate

In [14]:
y_pred = X_test["PREVIOUS"]
y_pred_train = X_train["PREVIOUS"]
print("# Baseline Model")
calcMetrics(y_train, y_pred_train, y_test, y_pred)

# Baseline Model
Root Mean Squared Error Train: 15.031755934972722
Root Mean Squared Error Test: 15.045528971082346


K-fold cross validation

In [15]:
# from sklearn.model_selection import KFold

# kf = KFold(n_splits=4)

# rmse_regression = []
# rmse_baseline = []
# accuracy_up_down = []
# for train_index, test_index in kf.split(X):
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
#     regressor = RandomForestRegressor(n_jobs=-1, random_state=42,n_estimators=resultado_gp.x[0], 
#                                   max_depth=resultado_gp.x[1],min_samples_leaf=resultado_gp.x[2],max_features=resultado_gp.x[3])

#     regressor.fit(X_train,y_train)
    
#     rmse_regression.append(
#         np.sqrt(metrics.mean_squared_error(regressor.predict(X_test),y_test))
#     )
    
#     up_df = pd.DataFrame({"Pred": regressor.predict(X_test), "Real": y_test, "Previous": X_test["PREVIOUS"]})
#     up_df["UP"] = up_df["Previous"] < up_df["Real"]
#     up_df["UP_PRED"] = up_df["Previous"] < up_df["Pred"]
#     up_df["UP"] = up_df["UP"].astype(int)
#     up_df["UP_PRED"] = up_df["UP_PRED"].astype(int)
#     accuracy_up_down.append(metrics.accuracy_score(up_df["UP"], up_df["UP_PRED"]))
    
#     rmse_baseline.append(
#         np.sqrt(metrics.mean_squared_error(X_test["PREVIOUS"],y_test))
#     )
    
# print("Mean RMSE for the regression model was:", np.mean(rmse_regression))
# print("Mean accuracy (up/down) for the regression model was:", np.mean(accuracy_up_down))
# print("Mean RMSE for the baseline model was:", np.mean(rmse_baseline))